## Qwen-2 is Alibaba's latest series of large language models (LLMs) designed for advanced natural language processing tasks.

In [1]:
!pip install accelerate gTTS gradio transformers

* accelerate : python lib helps with optimisation tech, run LLM faster
* gTTS : python lib for text to speed
* gradio : python lib , help yo build user interfaces
* transformers : python lib for train , fine-tune, inferance the model

In [2]:
from transformers import AutoModelForCausalLM, AutoTokenizer
from gtts import gTTS
import torch
import gradio as gr

#AutoToken = Encodes and decodes text for use with language models.
#AutoModelFor  = Loads pre-trained causal language models like GPT, Qwen,
  # or similar architectures with minimal configuration

In [3]:
device = "cuda" if torch.cuda.is_available() else "cpu" #if gpu is avaiable then we use it

In [4]:
langugage_model_name = "Qwen/Qwen2-1.5B-Instruct"   #installing the model
language_model = AutoModelForCausalLM.from_pretrained(langugage_model_name, device_map="auto", torch_dtype="auto")
tokenizer = AutoTokenizer.from_pretrained(langugage_model_name)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


model.safetensors:   0%|          | 0.00/3.09G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/242 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.29k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.78M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.67M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/7.03M [00:00<?, ?B/s]

In [5]:
def process_input(input_text, action):      #func
    if action == "Translate to English":
        prompt = f"Please translate the following text into English：{input_text}"
        lang = "en"
    elif action == "Translate to Chinese":
        prompt = f"Please translate the following text into Chinese：{input_text}"
        lang = "zh-cn"
    elif action == "Translate to Spanish":
        prompt = f"Please translate the following text into Spanish:{input_text}"
        lang = "sp"
    elif action == "Translate to French":
        prompt = f"Please translate the following text into French：{input_text}"
        lang = "fr"
    elif action == "Translate to Japanese":
        prompt = f"Please translate the following text into Japanese：{input_text}"
        lang = "ja"
    else:
        prompt = input_text
        lang = "en"
                                                          #role , Assistment
    messages = [
        {"role": "system", "content": "You are a helpful AI assistant."},
        {"role": "user", "content": prompt}
    ]
    text = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True
    )
    model_inputs = tokenizer([text], return_tensors="pt").to(device)

    generated_ids = language_model.generate(           #assine ids
        model_inputs.input_ids,
        max_new_tokens=512
    )
    generated_ids = [
        output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
    ]

    output_text = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]       #decoding for getting output
    return output_text, lang


In [6]:
def text_to_speech(text, lang):                    #converting text to speed
    tts = gTTS(text=text, lang=lang)
    filename = "output_audio.mp3"
    tts.save(filename)
    return filename

In [7]:
def handle_interaction(input_text, action):                         # func to store output
    output_text, lang = process_input(input_text, action)
    audio_filename = text_to_speech(output_text, lang)
    return output_text, audio_filename

In [8]:
action_options = [                 #define func
    "Translate to English",
    "Translate to Chinese",
    "Translate to Spanish",
    "Translate to French",
    "Translate to Japanese",
    "Chat"
  ]


Gradio App

In [9]:
iface = gr.Interface(             #making interface
    fn=handle_interaction,
    inputs=[
        gr.Textbox(label="input Text"),
        gr.Dropdown(action_options, label="select action")
    ],

    outputs=[
        gr.Textbox(label="output text"),
        gr.Audio(label="output audio")
    ],
    title="Translation and Chat App using AI",
    description="Translate input text or chat based on the selected action.",
    theme= "gradio/soft"
)

themes/theme_schema@0.0.3.json:   0%|          | 0.00/13.1k [00:00<?, ?B/s]

In [10]:
if __name__ == "__main__":                        #calling Gradio
    iface.launch(share=True)

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://85e5869394787feed5.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
